In [1]:
import pandas as pd
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from datetime import timedelta

def yaml_to_csv(input_folder, output_file):
    
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    combined_data = []

    for month_path in Path(input_folder).iterdir():
        if not month_path.is_dir():
            continue

        for file_path in month_path.glob('*.yaml'):
            with open(file_path, 'r') as file:
                data = yaml.safe_load(file)
                combined_data.append(pd.DataFrame(data))

    if combined_data:
        combined_df = pd.concat(combined_data, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f"Data saved to {output_file}")
    else:
        print("No data found.")

if __name__ == "__main__":
    input_folder = "D:/projects/stock/data"
    output_file = "D:/projects/stock/outputcsv/combined.csv"

    yaml_to_csv(input_folder, output_file)

Data saved to D:/projects/stock/outputcsv/combined.csv


In [19]:
import pandas as pd
from pathlib import Path
import os

def analyse_data():
    file_path = Path("D:/projects/stock/output/combined.csv")
    df = pd.read_csv(file_path)

    # Ensure 'date' is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year

    # Group by Ticker and year for yearly analysis
    yearly_data = df.groupby(['Ticker', 'year']).agg(
        first_open=('open', 'first'),
        last_close=('close', 'last')
    ).reset_index()

    # Calculate yearly return
    yearly_data['yearly_return'] = ((yearly_data['last_close'] - yearly_data['first_open']) / 
                                    yearly_data['first_open']) * 100

    # Identify top 10 green and red stocks
    top_10_green = yearly_data.nlargest(10, 'yearly_return')
    top_10_red = yearly_data.nsmallest(10, 'yearly_return')

    # Market summary
    green_stocks = (yearly_data['yearly_return'] > 0).sum()
    red_stocks = (yearly_data['yearly_return'] <= 0).sum()

    # Volatility analysis
    df['daily_return'] = df.groupby('Ticker')['close'].pct_change()
    volatility = df.groupby('Ticker')['daily_return'].std().reset_index()
    volatility.columns = ['Ticker', 'Volatility']
    volatility = volatility.nlargest(10, 'Volatility')

    # Calculate cumulative return
    df['cumulative_return'] = df.groupby('Ticker')['daily_return'].cumsum()

    # Calculate monthly returns
    df['month'] = df['date'].dt.to_period('M')
    monthly_data = df.groupby(['Ticker', 'month']).agg(
        open=('open', 'first'),
        close=('close', 'last')
    ).reset_index()
    monthly_data['monthly_return'] = ((monthly_data['close'] - monthly_data['open']) / 
                                      monthly_data['open']) * 100

    # Save results to CSV
    if not os.path.exists('outputcsv'):
        os.makedirs('outputcsv')

    top_10_green.to_csv('outputcsv/top_10_green_stocks.csv', index=False)
    top_10_red.to_csv('outputcsv/top_10_red_stocks.csv', index=False)
    print("Top 10 green and red stocks saved.")

    return df, yearly_data, top_10_green, top_10_red, volatility, monthly_data

# Call the function and unpack returned data
df, yearly_data, top_10_green, top_10_red, volatility, monthly_data = analyse_data()

# Merge yearly_data with df
df1 = df.merge(yearly_data, on=['Ticker'], how='left')

# Merge volatility
df1 = df1.merge(volatility, on='Ticker', how='left')

# Merge monthly data
df1 = df1.merge(monthly_data, on=['Ticker'], how='left')

# Save merged DataFrame to CSV
df1.to_csv('outputcsv/df1.csv', index=False)

print(df1.head())


Top 10 green and red stocks saved.


PermissionError: [Errno 13] Permission denied: 'outputcsv/df1.csv'

In [8]:
df1.head()

,Ticker,close_x,date,high,low,month,open_x,volume,year,daily_return,cumulative_return,first_open,last_close,yearly_return,Volatility,open_y,close_y,monthly_return
0,SBIN,602.95,2023-10-03 05:30:00,604.9,589.60,2023-10,596.6,15322196,2023,NaN,NaN,596.6,642.05,7.618170,NaN,596.6,565.55,-5.204492
1,BAJFINANCE,7967.60,2023-10-03 05:30:00,7975.5,7755.00,2023-10,7780.8,944555,2023,NaN,NaN,7780.8,7327.75,-5.822666,NaN,7780.8,7492.65,-3.703347
2,TITAN,3196.25,2023-10-03 05:30:00,3212.5,3114.40,2023-10,3148.8,1007308,2023,NaN,NaN,3148.8,3675.45,16.725419,NaN,3148.8,3189.65,1.297320
3,ITC,439.75,2023-10-03 05:30:00,442.9,439.25,2023-10,441.0,7481883,2023,NaN,NaN,441.0,462.10,4.784580,NaN,441.0,428.40,-2.857143
4,TCS,3513.85,2023-10-03 05:30:00,3534.2,3480.10,2023-10,3534.2,1948148,2023,NaN,NaN,3534.2,3793.40,7.334050,NaN,3534.2,3368.75,-4.681399


In [10]:

missing_data_summary = df.isnull().sum()
print("Missing data summary:")
print(missing_data_summary)

# Count the number of data points for each Ticker
ticker_counts = df['Ticker'].value_counts()
print("\nData points per Ticker:")
print(ticker_counts.head())

# Calculate the daily return and inspect for missing values
df['daily_return'] = df.groupby('Ticker')['close'].pct_change()
missing_daily_return = df[df['daily_return'].isnull()]
print("\nTickers with missing daily returns:")
print(missing_daily_return['Ticker'].unique())


Missing data summary:
Ticker                0
close                 0
date                  0
high                  0
low                   0
month                 0
open                  0
volume                0
year                  0
daily_return         50
cumulative_return    50
dtype: int64

Data points per Ticker:
Ticker
SBIN          284
BAJFINANCE    284
TITAN         284
ITC           284
TCS           284
Name: count, dtype: int64

Tickers with missing daily returns:
['SBIN' 'BAJFINANCE' 'TITAN' 'ITC' 'TCS' 'LT' 'TATACONSUM' 'RELIANCE'
 'HCLTECH' 'JSWSTEEL' 'ULTRACEMCO' 'POWERGRID' 'INFY' 'TRENT' 'BHARTIARTL'
 'TATAMOTORS' 'WIPRO' 'TECHM' 'NTPC' 'HINDUNILVR' 'APOLLOHOSP' 'M&M'
 'GRASIM' 'ICICIBANK' 'ADANIENT' 'ADANIPORTS' 'BEL' 'BAJAJFINSV'
 'EICHERMOT' 'COALINDIA' 'MARUTI' 'INDUSINDBK' 'ASIANPAINT' 'TATASTEEL'
 'HDFCLIFE' 'DRREDDY' 'SUNPHARMA' 'KOTAKBANK' 'SHRIRAMFIN' 'NESTLEIND'
 'ONGC' 'CIPLA' 'BPCL' 'BRITANNIA' 'SBILIFE' 'HINDALCO' 'HEROMOTOCO'
 'AXISBANK' 'HDFCBANK' 'B

In [12]:
df.tail()

,Ticker,close,date,high,low,month,open,volume,year,daily_return,cumulative_return
14195,HINDALCO,652.10,2024-11-22 05:30:00,657.3,647.40,2024-11,652.95,3586402,2024,0.006250,0.361646
14196,HEROMOTOCO,4794.10,2024-11-22 05:30:00,4808.4,4742.00,2024-11,4778.40,394712,2024,0.005506,0.502506
14197,AXISBANK,1142.40,2024-11-22 05:30:00,1147.9,1127.55,2024-11,1136.65,16687505,2024,0.002853,0.127365
14198,HDFCBANK,1745.60,2024-11-22 05:30:00,1754.3,1729.55,2024-11,1743.55,12386806,2024,0.002527,0.172242
14199,BAJAJ-AUTO,9481.65,2024-11-22 05:30:00,9602.9,9444.10,2024-11,9545.00,619823,2024,-0.002457,0.681613
